In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pandas as pd
import re

class PreprocessingText:
    def _init_(self):
        self.slang_list = [kamus.strip('\n').strip('\r') for kamus in open('/slang.txt')]
        self.stopword_list = [line.strip('\n')for line in open('/stop.txt')]
        
    # Case Folding
    def CaseFolding(self, tweets):
        tweets = tweets.lower()
        
        return tweets
    
    # Cleansing
    def Cleansing(self, tweets):
        # Menghilangkan mention, hashtag, character reference
        tweets = re.sub('[@#&][A-Za-z0-9_]+'," ", tweets)

        # Menghilangkan Tautan
        tweets = re.sub("\w+:\/\/\S+"," ", tweets)

        # Menghilangkan tanda baca
        tweets = re.sub('[()!?;,]', ' ', tweets)
        tweets = re.sub('\[.*?\]',' ', tweets)

        # Menghilangkan tanda selain huruf
        tweets = re.sub("[^a-z]"," ", tweets)

        # Menghilangkan spasi lebih dari 1
        tweets = ' '.join(tweets.split())
        
        return tweets
    
    # Replace Slangwords
    def Slangwords(self, tweets):
        for line in self.slang_list:
            slangword = line.strip().split(":")
            if slangword[0] in tweets.split(" "):
                tweets = tweets.replace(slangword[0] + " ", slangword[1] + " ")
                tweets = tweets.replace(" " + slangword[0], " " + slangword[1])
                
        return tweets
    
    # Stopwords Removal
    def StopwordsRemoval(self, tweets):
        stopwords_list = list(map(lambda x:x.strip(),list(self.stopword_list)))
        tweets = tweets.split()
        tweets = [w for w in tweets if not w in stopwords_list]
        tweets = " ".join(word for word in tweets)
        
        return tweets
    
    # Tokenization
    def Tokenization(self, tweets):
        tweets = tweets.split()
        
        return tweets
    
    # Normalization (Stemming)
    def Stemming(self, tweets):
        factory = StemmerFactory()
        stemmer = factory.create_stemmer()
        do = []
        for w in tweets:
            dt = stemmer.stem(w)
            do.append(dt)
        d_clean = []
        d_clean = " ".join(do)
        print(d_clean)
        
        return d_clean
    
    def Stemming1(self, tweets):
        stemmer = StemmerFactory.create_stemmer()
        tweets = stemmer.stem(tweets)
        print(tweets)

        return tweets

In [ ]:
df = pd.read_csv("Final_Sebelum_Kanjuruhan.csv", sep=';', encoding='unicode_escape')
df

In [ ]:
%%time
pretext = PreprocessingText()

# ======================================================================================================================

df_casefolding = df.copy()
df_casefolding['tweet'] = df_casefolding['tweet'].apply(pretext.CaseFolding)

df_cleansing = df_casefolding.copy()
df_cleansing['tweet'] = df_cleansing['tweet'].apply(pretext.Cleansing)
# df_cleansing = df_cleansing.drop_duplicates(subset='tweet')

df_slangwords = df_cleansing.copy()
df_slangwords['tweet'] = df_slangwords['tweet'].apply(pretext.Slangwords)

df_stopwords = df_slangwords.copy()
df_stopwords['tweet'] = df_stopwords['tweet'].apply(pretext.StopwordsRemoval)

df_token = df_stopwords.copy()
df_token['tweet'] = df_token['tweet'].apply(pretext.Tokenization)

df_stemming = df_token.copy()
# df_stemming['tweet'] = df_stemming['tweet'].apply(pretext.Stemming)

dataAll = {'Raw Data': df, 'Case Folding': df_casefolding, 'Cleansing': df_cleansing, 
           'Normalization': df_slangwords, 'StopWords Removal': df_stopwords, 'Stemming': df_stemming,
           'Tokenization': df_token}

writer = pd.ExcelWriter('./kanjuruhan_sesudah_Preprocessed.xlsx', engine='xlsxwriter', mode='w')

for data_sheet in dataAll.keys():
    dataAll[data_sheet].to_excel(writer, sheet_name=data_sheet, index=False)

writer.close()
writer.handles = None

print("\n\n\nData Preprocessed Saved!!!")

In [ ]:
import pandas as pd

data = pd.read_csv("Final_Sebelum_Kanjuruhan.csv", sep=';', encoding='unicode_escape')
# data = pd.read_csv("Final_Sesudah_Kanjuruhan.csv", sep=';', encoding='unicode_escape')
data

In [ ]:
tweets_positive = [t for t in data["label"] if t == "POSITIF"]
tweets_negative = [t for t in data["label"] if t == "NEGATIF"]

print("Hasil Sentimen Analisis")
print("POSITIF : ", len(tweets_positive), "({}%)".format(100*len(tweets_positive)/len(data)))
print("NEGATIF : ", len(tweets_negative), "({}%)".format(100*len(tweets_negative)/len(data)))
print("TOTAL : ", len(data))